# ADM Homework 3

## Point 1 Data Collection

#### 1.1. Get the list of master's degree courses

In [ ]:
#Import from 'getter.py' the function that this file need
from getter import create_list_pagesFINDaMASTERS, download_URLcourses

#Create a list ('pages') where it contains all the URL pages of the Masters
pages = create_list_pagesFINDaMASTERS()
#Create a file 'url-courses.txt' with all URL courses in each line
download_URLcourses(pages)

#### 1.2. Crawl master's degree pages

In [ ]:
#Import from 'crawler.py' the function that this file need
from crawler import check_file_exist, download_courses

#Put in variable 'txt_file' the path of file in which are the all URL courses
txt_file = 'url-courses.txt'
#Take the page number for which the download has not yet been made
npage = check_file_exist()

#Download HTML pages of the courses
download_courses(txt_file, npage)

#### 1.3 Parse downloaded pages

In [ ]:
#Import from 'parser_1.py' the function that this file need
from parser_1 import download_tsv, download_dataset_csv

#Download tsv file in the folder tsv
download_tsv()

#Initialize the file name
file_name = "dataset_courses.csv"
#Download dataset csv
download_dataset_csv(file_name)

## Point 2 Search Engine

In [1]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn nltk currency_converter

Note: you may need to restart the kernel to use updated packages.


In [2]:
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from currency_converter import CurrencyConverter
from nltk.stem import *
from collections import Counter
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
masters = pd.read_json("\dataset_courses.json")

[nltk_data] Downloading package stopwords to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 2.0 Preprocessing

This section of the question 2 is based on the process of cleaning the "description" attribute of the masters dataset, in particular, we removed the stopwords, the puntuation and implemented the WordNetLemmatizer() command.

In [4]:
masters

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,Develop in-depth knowledge of accounting and f...,September,Please see the university website for further ...,MSc,1 year full-time,Bath,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,Master's in Marketing and Brand Management,NHH Norwegian School of Economics,MSc,Full time,"Innovating, building and managing strong brand...",August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...
6002,Masters in Mathematical Finance,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OBJECTIVESThe Masters in Mathematical Finance ...,September,International Students: 6750€ UE Students: 5750€,MSc,2 Years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...
6003,Masters in Monetary and Financial Economics,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OVERVIEWThe Master's in Monetary and Financial...,September,International Students: 7000€ UE Students: 5750€,MSc,2 years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...
6004,Master's in Strategy and Management,NHH Norwegian School of Economics,MSc,Full time,The study of strategy is fundamentally about u...,August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...


In [5]:
# 2.0.0 Tokenize, remove stopwords and punctuation, and lemmatize

# i chose to use lemmatize to be more thorough and score better in parts 2 and 3

stop_words = set(stopwords.words('english'))

def tok_punct_and_stop(text):
    if text is None or pd.isnull(text):
        return []  # Return an empty list if the input is None or NaN
    desc_tok = word_tokenize(text)
    desc_tok_nostop_nopunct = [x for x in desc_tok if x.lower() not in stop_words and x not in string.punctuation]
    lemmed_desc = [lemmatizer.lemmatize(x) for x in desc_tok_nostop_nopunct]
    return lemmed_desc

masters["cleaned_description"] = masters["description"].apply(tok_punct_and_stop)


In [6]:
masters

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,cleaned_description
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[3D, visualisation, animation, play, role, man..."
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Businesses, government, rely, sound, financia..."
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,Develop in-depth knowledge of accounting and f...,September,Please see the university website for further ...,MSc,1 year full-time,Bath,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Develop, in-depth, knowledge, accounting, fin..."
3,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Accounting, Accountability, Financial, Manage..."
4,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Embark, professional, accounting, career, aca..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,Master's in Marketing and Brand Management,NHH Norwegian School of Economics,MSc,Full time,"Innovating, building and managing strong brand...",August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Innovating, building, managing, strong, brand..."
6002,Masters in Mathematical Finance,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OBJECTIVESThe Masters in Mathematical Finance ...,September,International Students: 6750€ UE Students: 5750€,MSc,2 Years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...,"[OBJECTIVESThe, Masters, Mathematical, Finance..."
6003,Masters in Monetary and Financial Economics,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OVERVIEWThe Master's in Monetary and Financial...,September,International Students: 7000€ UE Students: 5750€,MSc,2 years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...,"[OVERVIEWThe, Master, 's, Monetary, Financial,..."
6004,Master's in Strategy and Management,NHH Norwegian School of Economics,MSc,Full time,The study of strategy is fundamentally about u...,August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...,"[study, strategy, fundamentally, understanding..."


In [8]:
c = CurrencyConverter()


# new column to store standardized fees in euros
masters["fees_EUR"] = ""

def currency_format(text, country):
    text = str(text)
    if "Please see the university website for further information on fees for this course." in text or text == "" or text == "nan" or text is None:
        return 0.0
    # find all the numbers that refer to tuition fees
    else:
        numbers = re.findall(r"(?:[£$€]\s*)?\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?", text)
        numbers = [float(number.replace(",", "").replace("£", "").replace("$", "").replace("€", "").replace(".", "")) for number in numbers if number not in ["2023", "2024", "2025"]]
        amount = max(numbers) if numbers else None # keep only the maximum value to convert

        if amount is None:
            return 0.0 
        else:
            if amount < 10:
                return None     
            
            if country == "United Kingdom":  # check the country to minimzie text filtering to find the different ways GBP was reported
                fee = c.convert(amount, "GBP", "EUR") 
            elif "USD" in text: # the other currencies are usually reported in a standard way
                fee = c.convert(amount, "USD", "EUR")
            elif "CZK" in text:
                fee = c.convert(amount, "CZK", "EUR")
            elif "HK" in text:
                fee = c.convert(amount, "HKD", "EUR")
            elif "JPY" in text:
                fee = c.convert(amount, "JPY", "EUR")
            elif "SEK" in text:
                fee = c.convert(amount, "SEK", "EUR")
            else:
                fee = amount
            if int(fee) < 50:
                return 0.0
            
            # the code below is to fix the issues found when replacing commas and dots, 
            # where for example: 20.000,00 became 200000, instead of 20000(obserbved threshhold value)
            else:
                
                if fee > 200000:
                    return (round(fee, 1)/10)
                else: 
                    return round(fee, 1)

for i in tqdm(range(len(masters))):
    masters.loc[i, "fees_EUR"] = currency_format(masters.loc[i, "fees"], masters.loc[i, "country"])

masters

100%|██████████| 6006/6006 [00:00<00:00, 8367.88it/s]


,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,cleaned_description,fees_EUR
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[3D, visualisation, animation, play, role, man...",0.0
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Businesses, government, rely, sound, financia...",39935.6
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,Develop in-depth knowledge of accounting and f...,September,Please see the university website for further ...,MSc,1 year full-time,Bath,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Develop, in-depth, knowledge, accounting, fin...",0.0
3,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Accounting, Accountability, Financial, Manage...",0.0
4,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Embark, professional, accounting, career, aca...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,Master's in Marketing and Brand Management,NHH Norwegian School of Economics,MSc,Full time,"Innovating, building and managing strong brand...",August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...,"[Innovating, building, managing, strong, brand...",0.0
6002,Masters in Mathematical Finance,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OBJECTIVESThe Masters in Mathematical Finance ...,September,International Students: 6750€ UE Students: 5750€,MSc,2 Years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...,"[OBJECTIVESThe, Masters, Mathematical, Finance...",675.0
6003,Masters in Monetary and Financial Economics,University of Lisbon,ISEG - Lisbon School of Economics and Management,Part time,OVERVIEWThe Master's in Monetary and Financial...,September,International Students: 7000€ UE Students: 5750€,MSc,2 years,Lisbon,Portugal,On Campus,https://www.findamasters.com/masters-degrees/c...,"[OVERVIEWThe, Master, 's, Monetary, Financial,...",700.0
6004,Master's in Strategy and Management,NHH Norwegian School of Economics,MSc,Full time,The study of strategy is fundamentally about u...,August,Please see the university website for further ...,MSc,2 years full time,Bergen,Norway,On Campus,https://www.findamasters.com/masters-degrees/c...,"[study, strategy, fundamentally, understanding...",0.0


### 2.1 Conjunctive Query

This code creates an index for words, mapping the unique words from the ceaned_description attribute; the identifires are assigned in an increasing order staring from 1.

In [9]:
vocabulary = Counter(reduce(lambda x, y: x + y, masters.cleaned_description.values)).keys()
index = {}
unique_id = 1
for word in list(vocabulary):
  index[word] = unique_id
  unique_id+=1
index

{'3D': 1,
 'visualisation': 2,
 'animation': 3,
 'play': 4,
 'role': 5,
 'many': 6,
 'area': 7,
 'popularity': 8,
 'medium': 9,
 'keep': 10,
 'growing': 11,
 'Digital': 12,
 'provides': 13,
 'eye-catching': 14,
 'special': 15,
 'effect': 16,
 '21st': 17,
 'century': 18,
 "'s": 19,
 'favourite': 20,
 'film': 21,
 'television': 22,
 'show': 23,
 'design': 24,
 'also': 25,
 'essential': 26,
 'everyday': 27,
 'work': 28,
 'everything': 29,
 'computer': 30,
 'game': 31,
 'development': 32,
 'online': 33,
 'virtual': 34,
 'world': 35,
 'industrial': 36,
 'marketing': 37,
 'product': 38,
 'architecture.GCU': 39,
 'programme': 40,
 'Design': 41,
 'Virtual': 42,
 'Environments': 43,
 'help': 44,
 'develop': 45,
 'skill': 46,
 'thrive': 47,
 'successful': 48,
 'career': 49,
 'visual': 50,
 'designer': 51,
 'practical': 52,
 'career-focused': 53,
 'oriented': 54,
 'towards': 55,
 'current': 56,
 'industry': 57,
 'need': 58,
 'technology': 59,
 'practice': 60,
 'prior': 61,
 'knowledge': 62,
 'req

In [10]:
terms = pd.DataFrame(data=list(vocabulary), columns=['term'])
terms['documents'] = terms.term.apply(lambda item: list(masters.loc[masters.cleaned_description.apply(lambda row: item in row)].index))

In [11]:
def find_word_in_documents(word):
    filtered_rows = terms[terms['term'] == word]['documents'].values[0]
    result_df = masters.loc[filtered_rows, ['courseName', 'universityName', 'description', 'url']]
    return result_df

word_to_search = "account"
f_n_words = find_word_in_documents(word_to_search)
f_n_words

,courseName,universityName,description,url
225,International Tourism and Event Management MSc,Fresenius University of Applied Sciences,﻿YOUR INTERNATIONAL TOURISM & EVENT MANAGEMENT...,https://www.findamasters.com/masters-degrees/c...
355,MSc Finance and Accounting,University of Sheffield,This course takes you inside the world of acco...,https://www.findamasters.com/masters-degrees/c...
655,Actuarial Science - MSc,Xi’an Jiaotong-Liverpool University,An MSc Actuarial Science qualification will pr...,https://www.findamasters.com/masters-degrees/c...
727,Advanced Clinical Practice - MSc,University of Suffolk,IntroductionThis programme of study has been d...,https://www.findamasters.com/masters-degrees/c...
757,Advanced Clinical Practice (Postgraduate Certi...,University of Gloucestershire,This programme enables experienced healthcare ...,https://www.findamasters.com/masters-degrees/c...
1258,Applied Social Research MSc,University of Stirling,This Masters course provides you with the skil...,https://www.findamasters.com/masters-degrees/c...
2523,Criminological Research MSc,University of Stirling,This course provides you with expert training ...,https://www.findamasters.com/masters-degrees/c...
3809,Financial Technology with Data Science - MSc,University of Bristol,"From crowdfunding to cryptocurrencies, and fro...",https://www.findamasters.com/masters-degrees/c...
3876,Forensic Accounting (Distance Learning) (2 Yea...,University of Portsmouth,OverviewExplore financial crime investigations...,https://www.findamasters.com/masters-degrees/c...
3877,Forensic Accounting (Distance Learning) (3 Yea...,University of Portsmouth,OverviewExplore financial crime investigations...,https://www.findamasters.com/masters-degrees/c...


This function represents a mechanism that extracts all the documents in which is present the word inserted as input. It operates on the term dataframe looking for the rows related to the word inserted, so when the term matches the input word inserted. The output of it is a list of documents associated with this word.

### 2.2 Conjunctive Query and Ranking Score

This code applies the Term Frequency-Inverse Document Frequency (TF-IDF) vectorization technique on the "description" column of the "masters" DataFrame. After that inverted indexes are created for each term in the TF-IDF matrix, associating each term with a list of tuples containing document indices where the term appears and the corresponding TF-IDF values.
Given an input 'word_to_search,' this code computes the TF-IDF vector representation for the input word. It then calculates the cosine similarity between the TF-IDF vector of the input word and the entire TF-IDF matrix, obtained from the 'masters' DataFrame, representing the similarity of the input word to each document in the dataset. 
Finally the output of it will have 5 columns courseName, universityName, description, url and Simiarity for the highiest courses in terms of similarity.

In [16]:
tfidf = TfidfVectorizer()
masters["cleaned_description"].fillna("", inplace=True)
results = tfidf.fit_transform(masters.description)
termss = tfidf.get_feature_names_out()

AttributeError: 'NoneType' object has no attribute 'lower'

In [17]:
tfidf_matrix = results
feature_names = termss

NameError: name 'results' is not defined

In [51]:
def create_inverted_indexes(tfidf_matrix, features_names):
    inverted_indexes = defaultdict(list)
    for term_id, term in enumerate(features_names):
        term_indices = tfidf_matrix[:, term_id].nonzero()[0]
        for document_index in term_indices:
            tfidf_value = tfidf_matrix[document_index, term_id]
            inverted_indexes[term_id].append((document_index, tfidf_value))
    return inverted_indexes

inverted_indexes = create_inverted_indexes(tfidf_matrix, feature_names)

In [52]:
term_to_check = "account"
term_id_to_check = np.where(feature_names == term_to_check)[0][0]
print(f"{term_to_check}: {inverted_indexes[term_id_to_check]}")

account: [(727, 0.15363408723768698), (757, 0.18708430618423805), (1258, 0.26219759522160035), (2523, 0.1512623859524864), (4089, 0.14737202512821132), (4413, 0.19764568695355594), (4445, 0.1771873204808616), (4543, 0.20198336577935208), (4544, 0.15996273806643288), (5295, 0.22530476321014897), (5319, 0.21621849305860433), (5556, 0.25905364219826227), (5998, 0.22335657339725876)]


In [53]:
word_to_search = 'advanced knowledge'
word_tfidf = tfidf.transform([word_to_search])
cosine_s = cosine_similarity(word_tfidf, results).flatten()
point_2_2df = pd.DataFrame({
    'courseName': masters['courseName'],
    'universityName': masters['universityName'],
    'description': masters['description'],
    'url': masters['url'],
    'Similarity': cosine_s})
point_2_2df = point_2_2df.sort_values(by='Similarity', ascending = False)
point_2_2df.head(10)

,courseName,universityName,description,url,Similarity
977,Advanced Software Engineering MSc,King’s College London,Our Advanced Software Engineering MSc is an ad...,https://www.findamasters.com/masters-degrees/c...,0.412349
769,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for lea...,https://www.findamasters.com/masters-degrees/c...,0.361744
824,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,https://www.findamasters.com/masters-degrees/c...,0.344850
856,Advanced Healthcare Practice MSc,Liverpool John Moores University,Developed in response to the current Advanced ...,https://www.findamasters.com/masters-degrees/c...,0.334779
776,Advanced Clinical Practice MSc/PGDip,Middlesex University,Our Advanced Clinical Practice (ACP) degree wi...,https://www.findamasters.com/masters-degrees/c...,0.331571
723,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,https://www.findamasters.com/masters-degrees/c...,0.320308
780,Advanced Clinical Practitioner - MSc,University of Sunderland,The MSc Advanced Clinical Practitioner is a hi...,https://www.findamasters.com/masters-degrees/c...,0.293375
854,Advanced Healthcare Practice - MSc,Cardiff University,Why study this courseOur MSc Advanced Healthca...,https://www.findamasters.com/masters-degrees/c...,0.283824
898,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,This course offers a broad range of advanced s...,https://www.findamasters.com/masters-degrees/c...,0.283423
964,Advanced Professional Practice (MSc),University of Gloucestershire,Our lecturers are research active experts who ...,https://www.findamasters.com/masters-degrees/c...,0.278472


## Point 3 Define a New Score

In this part we will be implementing an alternative, and more rigorous scoring function compared to the one presented in 2.2.

This function will be scoring the reuslts accordidng to the following criteria:

1- Description matching as before
2- Exact matches in course titles
3- Possible matches in geographical location
4- Possible tuition fee matching, preferring universities with tuition costs lower than any that may be specified in the query.
5- Possible matches in modality and administration


*Score 1: Description matching*

In [18]:
def description_score(query_tok):
    description_terms = masters["cleaned_description"]
    match_count = description_terms.apply(lambda x: sum(token in x for token in query_tok))
    return match_count

*Score 2: Exact Matches in Course Titles*

In [19]:
def title_exact_match_score(query_l):
    title_matches = masters[masters['courseName'].str.lower().isin(query_l)]['courseName'].tolist()
    return 1 if  title_matches else 0

*Score 3: Geographical location*

In [20]:
def location_score(query_l):
    matching_locations = masters[
    (masters['city'].str.lower().isin(query_l)) &
    (masters['country'].str.lower().isin(query_l))
]['city'].tolist()
    return 1 if matching_locations else 0

*Score 4: Tuition fees matching*

Only works if masters["fees"] is reported in the dataset

In [21]:
def fee_score(user_budget):
    if masters["fees_EUR"].any() > 0.0:
        under_budget = float(user_budget) >= masters["fees_EUR"].any()
    return 1 if under_budget else 0

Score 5: Administration score

In [22]:
def administration_score(query_l):
    admin_match = masters[masters["administration"].str.lower().isin(query_l)]["administration"].tolist()
    return 1 if admin_match else 0

Overall Score

Step 2: Scoring functions and weights

In [23]:
def overall_score(query, masters):
 
    query_tok = tok_punct_and_stop(query) #for budget and description matching
    query_l = word_tokenize(query) # for exact matches search
    for element in query_tok:
        if element.isnumeric():
            budget = element

    desc_score = description_score(query_tok)
    title_exact_match = title_exact_match_score(query_l)
    location_score_value = location_score(query_l)
    fee_score_value = fee_score(budget)
    admin_score_value = administration_score(query_l)
    #duration_score_value = duration_score(user_duration)

    total_score = (
        (desc_score * 0.6) +
        (title_exact_match * 0.6) +
        (location_score_value * 0.2) +
        (fee_score_value * 0.2) +
        (admin_score_value * 0.1)
    )

    return total_score

**Score**

In [24]:
user_query = "Management full time in London under 2000 online for 1 year"

score = overall_score(user_query, masters)


masters_ranked = pd.DataFrame({
    'courseName': masters['courseName'],
    'universityName': masters['universityName'],
    'description': masters['description'],
    'fees' : masters['fees_EUR'],
    'location' : masters["city"],
    'url': masters['url'],
    'score': score})

masters_ranked = masters_ranked.sort_values(by='score', ascending = False)
masters_ranked

,courseName,universityName,description,fees,location,url,score
4871,International Management - MSc,Audencia Business School,"About the CourseCUSTOMISABLE, INTERNATIONAL, F...",0.0,Nantes,https://www.findamasters.com/masters-degrees/c...,3.3
3160,Educational Leadership and Management (Distanc...,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7
4716,International Business Management - MSc/PGDip/...,"St Mary’s University, Twickenham",This degree focuses on the development of the ...,0.0,London,https://www.findamasters.com/masters-degrees/c...,2.7
3162,Educational Leadership and Management MSc,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7
2052,Clinical Education MSc/PgDip/PgCert,University of Exeter,OverviewLearn the craft of teaching from inter...,0.0,Exeter,https://www.findamasters.com/masters-degrees/c...,2.7
...,...,...,...,...,...,...,...
2955,Digital Marketing MSc,King’s College London,Do you want to pursue a career in digital mark...,0.0,London,https://www.findamasters.com/masters-degrees/c...,0.3
2956,Digital Marketing MSc,University of Exeter,OverviewFocus on the practical application of ...,0.0,Exeter,https://www.findamasters.com/masters-degrees/c...,0.3
2957,Digital Marketing MSc,Nottingham Trent University,Digital marketing is one of the fastest growin...,0.0,Nottingham,https://www.findamasters.com/masters-degrees/c...,0.3
2958,Digital Marketing MSc,Teesside University,Our MSc Digital Marketing gives you the skills...,0.0,Middlesbrough,https://www.findamasters.com/masters-degrees/c...,0.3


##  Point 4 Visualizing the most relevant MSc degrees

#### latitude and longitude informations in the csv are obtained with the geocoder opensource geocoding api.

In [ ]:
'''import geocoder

# Function to get latitude and longitude
def get_lat_lng(university_name):
    g = geocoder.osm(university_name)
    if g.ok:
        return g.latlng
    else:
        return None

# List of university names
university_names =  masters_ranked['universityName'].unique().tolist()
# Dictionary comprehension to create the desired dictionary
location_dict = {name: get_lat_lng(name) for name in university_names}

'''


#

In [ ]:
df = pd.read_csv('master_ranked_lat_lot.csv',index_col = 0).reset_index(drop=True)
df

,courseName,universityName,description,fees,location,url,score,latitude,longitude
0,International Management - MSc,Audencia Business School,"About the CourseCUSTOMISABLE, INTERNATIONAL, F...",0.0,Nantes,https://www.findamasters.com/masters-degrees/c...,3.3,47.247009,-1.552633
1,Educational Leadership and Management (Distanc...,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7,50.795245,-1.093553
2,International Business Management - MSc/PGDip/...,"St Mary’s University, Twickenham",This degree focuses on the development of the ...,0.0,London,https://www.findamasters.com/masters-degrees/c...,2.7,51.437068,-0.336095
3,Educational Leadership and Management MSc,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7,50.795245,-1.093553
4,Clinical Education MSc/PgDip/PgCert,University of Exeter,OverviewLearn the craft of teaching from inter...,0.0,Exeter,https://www.findamasters.com/masters-degrees/c...,2.7,50.736937,-3.534735
...,...,...,...,...,...,...,...,...,...
5985,Design for Food (Specializing Master),"POLI.design, Società consortile a responsabili...",The Specializing Master DESIGN FOR FOOD integr...,16500.0,Milan,https://www.findamasters.com/masters-degrees/c...,0.3,45.464194,9.189635
5986,Design for Architecture (Specializing Master),"POLI.design, Società consortile a responsabili...","Future designers, architects and engineers wil...",12000.0,Milan,https://www.findamasters.com/masters-degrees/c...,0.3,45.464194,9.189635
5987,Cyber Security (MSc),University of Rennes 1,Cybersecurity has been called the greatest thr...,0.0,Rennes,https://www.findamasters.com/masters-degrees/c...,0.3,48.111339,-1.680020
5988,Econometrics (MSc),Erasmus School of Economics,Would you like to acquire the skills to be abl...,16100.0,Rotterdam,https://www.findamasters.com/masters-degrees/c...,0.3,51.924442,4.477750


In [ ]:
import folium
import pandas as pd
import numpy as np


#legend rules
def get_color(fee):
    if fee == 0:
        return 'green'
    elif fee <= 5000:
        return 'blue'
    elif fee <= 10000:
        return 'orange'
    elif fee <= 20000:
        return 'red'
    else:
        return 'purple'

#prevent overlapping locations
def add_jitter(lat, lon, degree_of_jitter=0.001):
    return (lat + np.random.uniform(-degree_of_jitter, degree_of_jitter),
            lon + np.random.uniform(-degree_of_jitter, degree_of_jitter))

#creating map object.
map_center_lat = df['latitude'].mean()
map_center_long = df['longitude'].mean()
university_map = folium.Map(location=[map_center_lat, map_center_long], zoom_start=2)

#putting points on the map
for _, row in df.iterrows():
    jittered_lat, jittered_lon = add_jitter(row['latitude'], row['longitude'])
    
    folium.CircleMarker(
        location=[jittered_lat, jittered_lon],
        radius=5,
        popup=f"{row['universityName']} - {row['courseName']} - Fees: {row['fees']}",
        color=get_color(row['fees']),
        fill=True,
        fill_color=get_color(row['fees'])
    ).add_to(university_map)


#adding legend
legend_html = '''
<div style="position: fixed; 
     top: 50px; right: 50px; width: 200px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background: white; padding: 10px; border-radius: 5px;
     box-shadow: 0 0 15px rgba(0,0,0,0.4);">
     &nbsp; Cost Legend <br>
     &nbsp; Free/No Cost Found &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
     &nbsp; ≤ $5,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i><br>
     &nbsp; ≤ $10,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
     &nbsp; ≤ $20,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
     &nbsp; > $20,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i>
</div>
'''
university_map.get_root().html.add_child(folium.Element(legend_html))

#saving map as html
university_map.save('university_map.html')


##  Point 7 Algorithmic Question (AQ)

#### 1. Implement a code to solve the above mentioned problem.

**Preliminary assumption:**

We assume that Leonardo can have 0 hours of work in a day, so cases in which 0 is chosen among the range of hours are allowed
We assume that the largest range of hours can be [0,8], so a day of work can't have more than 8 hours (even without this constratin the algorithm would work as well)

**The idea behind it is the following one:**

- Given the 3 inputs, the machine calculates all the possible combinations of hours worked each day;
- After calcuating all the combinations, all those for which the sum of hours worked for the single range is different from the sum of hours worked are discarded, an example: given d=3, sumHours=5 and ranges: d1 [0,1], [0,2], [1,3], the computer will calculate all the possible combinations of hours of work, for example a combination can be d1=1, d2=1 and d3=3, in this case as long as the sum of these 3 element is exactly equal to sumHours, this combination will be kept; an alternavtive combination could have been: d1=1, d2=1 and d3=2, the sum is different from 5 and this combination will be discared;
- If there is a solution "YES" is printed together with the number of hours per each day (if there is more than 1 solution, just a random one is printed among all the right combinations), if there isn't a solution just "NO" is printed.

In [ ]:
#Import function
from itertools import product


#Define 'all_combinations' function
def all_combinations(d, sumHours, hours_range):

    #Initialize 'combinations' list
    combinations = []

    #Generate all possible combinations of hours worked on each day
    for hours in product(*[range(r[0], min(r[1] + 1),9) for r in hours_range]):
        #If 'sum(hours)' is equal to 'sumHours'...
        if sum(hours) == sumHours:
            #...then append 'hours' in 'combinations' list
            combinations.append(hours)

    #Return 'combinations'
    return combinations


#Define 'main' function
def main():

    #Ask 'd' and 'sumHours' in input
    d, sumHours = map(int, input("'d' and 'sumHours' separeted with a space: ").split())

    #Initialize 'hours_range' list
    hours_range = []

    #Ask for 'd' time...
    for i in range(d):
        #... 'd_range' list in two integer numbers
        d_range = list(map(int, input(f" day {i + 1} (please separa the number with a space): ").split()))
        #Append these in 'hours_range' list
        hours_range.append(d_range)

    #Create the all combinations
    combinations = all_combinations(d, sumHours, hours_range)

    #If combinations exists...
    if combinations:
        #...print 'YES' and print one of multiple combinations
        print("YES")
        print(" ".join(map(str, combinations[0])))
    
    #Else...
    else:
        #...print 'NO'
        print("NO")


#Start the function 'main'...
if __name__ == "__main__":
    main()

#### 2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

The 'time complexity' (Big O notation) for our algorithm is $O(n^2)$. This is because the Big O notation goes to analyze the worst case of the algorithm. In our case, the worst case is when the algorithm needs to calculate all possible combinations for our exercise. All the details on the calculation are provided in the main file of this repository.

In [ ]:
from itertools import product

def all_combinations(d, sumHours, hours_range):
    combinations = []                                                                                           #1
    for hours in product(*[range(r[0], min(r[1] + 1, 9)) for r in hours_range]):                                    #2^d
        if sum(hours) == sumHours:                                                                              #1
            combinations.append(hours)                                                                          #1
    return combinations

def main():
    d, sumHours = map(int, input("'d' and 'sumHours' separeted with a space: ").split())                        #1
    hours_range = []                                                                                            #1
    for i in range(d):                                                                                              #d
        d_range = list(map(int, input(f" day {i + 1} (please separa the number with a space): ").split()))      #1
        hours_range.append(d_range)                                                                             #1
    combinations = all_combinations(d, sumHours, hours_range)                                                       #2^d
    if combinations:                                                                                            #1
        print("YES")                                                                                            #1
        print(" ".join(map(str, combinations[0])))                                                                  #d
    else:
        print("NO")                                                                                             #1

if __name__ == "__main__":
    main()

#### 3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)

**ChatGPT** *: Let's analyze the time complexity of your code. The key factor here is the generation of all possible combinations of hours worked on each day using the product function from the itertools module. The time complexity of the product function is $O(n^r)$, where $n$ is the length of the input iterable, and $r$ is the number of input iterables (in this case, the number of days d).*


Comparing our response with ChatGPT's, the time complexity (the Big O Notation) seems to be the same, as $n$, the length of the input iterable, in our case is 2. 'r' as previously explained by ChatGPT, is equivalent to the number of days 'd'.

#### 4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

We believe that our code is mostly functional according the way we solved it (exlaination in point 1); probably a more optimal solution to this problem does exist, but assuming that we change the structure behind the code itself.